_Neural Data Analysis_

Lecturer: Prof. Dr. Philipp Berens, Dr. Alexander Ecker

Tutors: Sarah Strauss, Santiago Cadena

Summer term 2019

Due date: 2019-07-09, 9am 

Names: Guillem Boada, Peter El-Jiz, Ulzii

# Warning 

The code will not run with the default https://github.com/mackelab/poisson-gpfa. The library has been forked and heavy modifications have been performed. It can be found at https://github.com/peterjiz/poisson-gpfa

# Exercise sheet 8

In this exercise we are going to fit a latent variable model (Poisson GPFA) to both toy data and real data from monkey primary visual cortex.

## Preliminaries

1. Clone the poisson-GPFA repository from https://github.com/mackelab/poisson-gpfa and make sure that you have a directory ```../funs/```  in the folder with this notebook. The toolbox contains an implementation of the EM algorithm to fit the poisson-gpfa. For the desciption of the algorithm please refer to https://hooram.xyz/projects.html 

2. Download the data file ```nda_ex_8_data.mat``` from ILIAS and save it in a subfolder ```../data/```.

### C-Style Print Statements

In [1]:
import sys
def sprintf(format, *values):
    s = format % values 
    return s

def printf(format, *values):
    sys.stdout.write(format % values)

### Code Imports

In [2]:
import seaborn as sns
import funs.util as util
import funs.engine as engine 
import funs.cvalidate as cvalidate
import matplotlib.pyplot as plt
import numpy as np
import datamanager
import scipy.io as sio
import numpy as np
import matplotlib.gridspec as gridspec
import pdb
import numpy.matlib
import pandas as pd
from statsmodels.stats.moment_helpers import cov2corr

sns.set_style("whitegrid", {'axes.grid' : False})
sns.set_context('paper')
sns.set(rc={'image.cmap': 'bwr'})
% matplotlib inline

ModuleNotFoundError: No module named 'funs.cvalidate'

### Reload Module

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(engine)
importlib.reload(cvalidate)

## Task 1. Generate some toy data to test the poisson-GPFA code

We start by verifying our code on toy data. The cell below contains code to generate data for 30 neurons, 100 trials (1000 ms each) and 50ms bin size. The neurons' firing rate $\lambda_k$ is assumed to be a constant $d_k$ modulated by a one-dimensional latent state $x$, which is drawn from a Gaussian process:

$\lambda_k = \exp(c_kx + d_k)$

Each neuron's weight $c_k$ is drawn randomly from a normal distribution and spike counts are sampled from a Poisson distribution with rate $\lambda_k$.

Your task is to fit a Poisson GPFA model with one latent variable to this data (see `engine.PPGPFAfit`).

*Grading: 3 pts*

In [ ]:
# Initialize random number generator
#np.random.seed(10)
# Specify dataset & fitting parameters
xdim      =1#latent dimensionality to fit
ydim      = 30		 #number of neurons in the dataset
numTrials = 100		
trialDur  = 1000 # in ms
binSize   = 50	 # in ms
maxEMiter = 100		
dOffset   = 1.5	 # controls firing rate

# Sample from the model (make a toy dataset)
training_set  = util.dataset(
    seed      =345533,
    xdim      = xdim,
	ydim      = ydim,
	numTrials = numTrials,
	trialDur  = trialDur,
	binSize   = binSize,
	dOffset   = dOffset,
	fixTau 	  = True, 
	fixedTau  = np.array([0.2]),
	drawSameX = False) 

### Fit the model

In [ ]:
# Initialize parameters using Poisson-PCA
initParams = util.initializeParams(xdim, ydim, training_set)

# fit the model
fitToy = engine.PPGPFAfit(
    #todo
	experiment 		= training_set,
	initParams 		= initParams,
	inferenceMethod = 'laplace',
	EMmode 			= 'Batch',
	maxEMiter 		= maxEMiter
)

In [ ]:
#some useful functions
def allTrialsState(fit,p):
    """Reshape the latent signal and the spike counts"""
    x = np.zeros([p,0])
    for i in range(len(fit.infRes['post_mean'])):
        x = np.concatenate((x,fit.infRes['post_mean'][i]),axis=1)
    return x

def allTrialsX(training_set):
    """Reshape the ground truth 
    latent signal and the spike counts"""
    x_gt = np.array([])
    for i in range(len(training_set.data)):
        x_gt =  np.concatenate((x_gt,training_set.data[i]['X'][0]),axis = 0)
    return x_gt

### Plot the ground truth vs. inferred model
Verify your fit by plotting both ground truth and inferred parameters for:
1. weights C
2. biases d
3. spike counts covariance matrix
4. latent state x 

Note that the sign of fitted latent state and its weights are ambiguous (you can flip both without changing the model). Make sure you correct the sign for the plot if it does not match the ground truth.

In [ ]:
# All trials latent state vector
x = allTrialsState(fitToy,1)
x_gt = allTrialsX(training_set)

In [ ]:
C_gt = training_set.params['C']
d_gt = training_set.params['d']
tau_gt = training_set.params['tau']

C_est = fitToy.optimParams['C']
d_est = fitToy.optimParams['d']
tau_est = fitToy.optimParams['tau']

#### Weights & Biases

In [ ]:
# plot ground truth weights C, biases d and K(tau)
plt.suptitle('Ground truth weights C and biases d', fontsize=17)
training_set.plotParams()

In [ ]:
# plot fitted weights C, biases d and K(tau)
plt.suptitle('Inferred weights C and biases d',fontsize=17)
fitToy.plotOptimParams()

In [ ]:
plt.figure(figsize=(24, 12))

plt.subplot(2, 2, 1)
plt.title("Ground Truth C")
plt.plot(C_gt)

plt.subplot(2, 2, 2)
plt.title("Estimated C")
plt.plot(-C_est)

plt.subplot(2, 2, 3)
plt.title("Ground Truth d")
plt.plot(d_gt)

plt.subplot(2, 2, 4)
plt.title("Estimated d")
plt.plot(d_est)

### Covariances

In [ ]:
# spike counts covariance matrix
fitToy.plotCovAnalysis()

In [ ]:
plt.figure(figsize=(24, 6))
plt.subplot(1, 2, 1)
plt.title("Ground Truth Cov[y]")
plt.imshow(fitToy.E_yy_true_params)

plt.subplot(1, 2, 2)
plt.title("Estimated Cov[y]")
plt.imshow(fitToy.E_yy_optim_params)

Since $x$ is of dimensions $pT * 1$; to get all times for a trial, we need to index from $trial * (T=\text{max time bin})$ to $(trial+1)*(T=\text{max time bin})$

In [ ]:
# fill in plot here
plt.figure(figsize=(24, 6))
plt.subplot(1, 2, 1)
plt.title("Ground Truth x")
plt.plot(x_gt)

plt.subplot(1, 2, 2)
plt.title("Estimated x")
plt.plot(x.T)

In [ ]:
# fill in plot 
plt.figure(figsize=(24,12))
plt.suptitle('Ground Truth & Estimated x',fontsize=17)
for trial in range(0, 10):
    plt.subplot(2, 5, trial+1)
    plt.plot(x_gt[trial*20:(trial+1)*20],label='Ground Truth')
    plt.plot(-x.T[trial*20:(trial+1)*20],'r',label='Estimated')
    plt.title(sprintf("Trial %i", trial+1))
    plt.xlabel('Time Bins')
    plt.ylabel('Latent x')
    plt.legend()

## Task 2: Fit GPFA model to real data. 

We now fit the model to real data and cross-validate over the dimensionality of the latent variable.

*Grading: 2 pts*



### Load data

The cell below implements loading the data and encapsulates it into a class that matches the interface of the Poisson GPFA engine. You don't need to do anything here.

In [ ]:
class EckerDataset():
    """Loosy class"""
    def __init__(
        self,
        path = 'data/task8_data.mat',
        subject_id=0,
        ydim = 55,
        trialDur = 400,
        binSize = 20,
        numTrials = 100,
        ydimData = False,
        numTrData = True):
        
        T = binSize#int(trialDur/binSize)
        matdat = sio.loadmat(path)
        self.matdat = matdat
        data = []
        trial_durs = []
        for trial_id in range(numTrials):
            trial_time =matdat['spikeTimes'][:,trial_id][0] 
            trial_big_time = np.min(trial_time)
            trial_end_time = np.max(trial_time)
            trial_durs.append(trial_end_time - trial_big_time)         
        for trial_id in range(numTrials):
            Y = []
            spike_time = []
            data.append({
                'Y': matdat['spikeCounts'][:,:,trial_id],
                'spike_time': matdat['spikeTimes'][:,trial_id]})
        self.T = T
        self.trial_durs = trial_durs    
        self.data = data
        self.trialDur = trialDur
        self.binSize = binSize
        self.numTrials = numTrials
        self.ydim = ydim              
        util.dataset.getMeanAndVariance(self)
        util.dataset.getAvgFiringRate(self)
        util.dataset.getAllRaster(self)

In [ ]:
path = '../data/nda_ex_8_data.mat'
data = EckerDataset(path)

### Fit Poisson GPFA models and perform model comparison

Split the data into 80 trials used for training and 20 trials held out for performing model comparison. On the training set, fit models using one to five latent variables. Compute the performance of each model on the held-out test set.

Hint: You can use the `crossValidation` function in the Poisson GPFA package.

Optional: The `crossValidation` function computes the mean-squared error on the test set, which is not ideal. The predictive log-likelihood under the Poisson model would be a better measure, which you are welcome to compute instead.

In [ ]:
import scipy.optimize as op 
import funs.inference as inference 

def predictiveLogLikeErrorFunc(params, experiment):
    '''
    Performs leave-one-out prediction. 
    '''
    ydim, xdim = np.shape(params['C'])
    print('Performing cross validation with predictiveLogLikeErrorFunc...')
    y_pred_mode_all = []
    pred_err_mode = 0
    for tr in range(experiment.numTrials):
        y_pred_mode_tr = []
        for nrn in range(experiment.ydim):
            # Make params without neuron# nrn
            CwoNrn = np.delete(params['C'],nrn,0)
            dwoNrn = np.delete(params['d'],nrn,0)
            paramsSplit = {'C':CwoNrn, 'd':dwoNrn, 'tau':params['tau']}

            # Make params with only neuron# nrn
            C_nrn = params['C'][nrn]
            d_nrn = params['d'][nrn]

            # Make params big
            C_big, d_big = util.makeCd_big(paramsSplit,experiment.T)
            K_big, K = util.makeK_big(paramsSplit, experiment.trialDur, experiment.binSize)
            K_bigInv = np.linalg.inv(K_big)

            # Make data without neuron# nrn
            y = np.delete(experiment.data[tr]['Y'],nrn,0)
            ybar = np.ndarray.flatten(np.reshape(y, (experiment.ydim-1)*experiment.T))

            xInit = np.ndarray.flatten(np.zeros([xdim*experiment.T,1]))
            res = op.fmin_ncg(
                f = inference.negLogPosteriorUnNorm,
                x0 = xInit,
                fprime = inference.negLogPosteriorUnNorm_grad,
                fhess = inference.negLogPosteriorUnNorm_hess,
                args = (ybar, C_big, d_big, K_bigInv, xdim, experiment.ydim-1),
                disp = False,
                full_output = True)

            x_post_mode = np.reshape(res[0],[xdim,experiment.T])
            y_pred_mode_nrn = np.exp(C_nrn.dot(x_post_mode).T + d_nrn)
            pred_err_mode = pred_err_mode # + what needs to change - np.dot(experiment.data[tr]['Y'][nrn]-y_pred_mode_nrn,experiment.data[tr]['Y'][nrn]-y_pred_mode_nrn)
            y_pred_mode_tr.append(y_pred_mode_nrn)
        y_pred_mode_all.append(y_pred_mode_tr)
    y_pred_mode = np.asarray(y_pred_mode_all)
    pred_err_mode = pred_err_mode
    return y_pred_mode, pred_err_mode

In [ ]:
import importlib
importlib.reload(util)
importlib.reload(engine)
importlib.reload(inference)

# # your code here
cv = cvalidate.crossValidation(data,
        numTrainingTrials = 80,
        numTestTrials = 20,
        maxXdim = 5,
        maxEMiter = 3,
        batchSize = 5,
        inferenceMethod = 'laplace',
        learningMethod = 'batch') #, 
        #errorFunc = predictiveLogLikeErrorFunc)

### Plot the test error

Make a plot of the test error for the five different models. As a baseline, please also include the test error of a model without a latent variable. This is essentially the mean-squared error of a constant rate model (or Poisson likelihood if you did the optional part above).

In [ ]:
def mergeTrials(data):
    trialsAmt = data.numTrials
    neuronsAmt = data.ydim
    binsAmt = data.data[0]['Y'].shape[1]
    
    mergedTrials = np.zeros((trialsAmt, neuronsAmt, binsAmt))
    
    for t, trial in enumerate(data.data):
        mergedTrials[t, :, :] = trial['Y']
        
    return mergedTrials

# SPLIT THE DATA
train, test = util.splitTrainingTestDataset(data, numTrainingTrials=80, numTestTrials=20)

mergedTrainData = mergeTrials(train)
mergedTestData = mergeTrials(test)

# FIT
fit_0 = np.mean(mergedTrainData, axis=0)  # average over trials and bins

# COMPUTE ERROR
err_0 = ((fit_0-mergedTestData)**2).sum()

In [ ]:
errs = np.concatenate([[cv.err_0], cv.errs], axis=0)
plt.figure(figsize=(5,4))
# plt.axhline(errs[0], color='r', markersize=5,linewidth=2, label="Baseline")
#plt.plot(np.arange(0, 6), errs,'b.-',markersize=5,linewidth=2)
plt.scatter(0, errs[0], color='red', label="Baseline (Hacky)")
plt.scatter(0, err_0, color='orange', label="Baseline (Mean Squared)")
plt.scatter(np.arange(1, 6), errs[1:])
plt.xlabel('Latent Dimensionality')
plt.ylabel('Error')
plt.title('Latent Dimension vs. Prediction Error')
plt.grid(which='both')
plt.tight_layout()
plt.legend(loc="best")

## Task 3. Visualization: population rasters and latent state. Use the model with a single latent state. 

Create a raster plot where you show for each trial the spikes of all neurons as well as the trajectory of the latent state `x`. Sort the neurons by their weights `c_k`. Plot only the first 20 trials.

*Grading: 2 pts*

In [ ]:
def constructCMatrix(fits):
    C = np.zeros((np.shape(fits[0].optimParams['C'])[0], np.shape(fits)[0]))
    for i, fit in enumerate(fits):
        C[:, i] = fit.optimParams['C'][:, 0]
    return C

We are asked to use a model with a single latent state; in this case, we will just be working with C[:, 0]

In [ ]:
C_est_matrix = constructCMatrix(cv.fits)
c = C_est_matrix[:, 0]

x_fitted = np.array(cv.fits[0].infRes['post_mean']).reshape((80,20))
trialDur = data.trialDur
numBins = data.binSize
bin_length = trialDur/numBins
bin_midpoints = np.linspace(0.5*bin_length,trialDur-0.5*bin_length,numBins)

In [ ]:
neuronIndicesSortedByWeight = np.argsort(c)

In [ ]:
Y_alltrials = np.zeros((20, 55, 20)) #20 trials, 55 neurons, 20 time bins
spiketimes_alltrials = [] #np.zeros((20, 55)) #20 trials, 55 neurons, 

for trial in range(0, 20): 
    Y = data.data[trial]['Y'][neuronIndicesSortedByWeight]
    spiketimes = data.data[trial]['spike_time'][neuronIndicesSortedByWeight]
    
    Y_alltrials[trial, :, :] = Y 
    spiketimes_alltrials.append(spiketimes)
spiketimes_alltrials = np.array(spiketimes_alltrials)

In [ ]:
def plotRaster(trialsAmt, Y_all, spiketimes_all, scale = 1.0, spacing=1.0, flip=False):
    if (flip==False):
        flip = 1.0
    else: 
        flip = -1.0
                
    # For each direction 
    for trial in range(0, 20): 
        Y = Y_all[trial] 
        spiketimes = spiketimes_all[trial] 
        # Use different colors for odd and even 
        color = "red"
        alternateColor = "blue"
        if (trial % 2 == 0): 
            color = "blue"
            alternateColor = "red"
        
        normalizedY = Y[:, :] #/ (np.max(Y[:, :]) * 55)
        normalizedSpiketimes = spiketimes[:]
        plt.axhline(flip*spacing*trial*scale, color='black', markersize=0.1,linewidth=0.2)
        for neuron in range(0, 55):
            neuronY = normalizedY[neuron, :] / 55 # (np.max(normalizedY[neuron, :]) * 55)
            neuronSpiketimes = normalizedSpiketimes[neuron] / 55
            y = (flip*(spacing)*trial + flip*neuron/55 + (np.ones((np.shape(neuronSpiketimes)))))*scale
            plt.scatter(neuronSpiketimes[:], y, c=color, s = 2) 

    
    yticks = flip*np.arange(0, spacing*(trialsAmt), spacing)*scale
    labels = np.array(yticks/(spacing) + 1, dtype=int)
    plt.yticks(yticks, labels, fontsize=12)
    plt.ylim(0, (trialsAmt)*spacing*scale)
#     plt.xlim(-1, 21)
#     plt.xticks(np.arange(0, 20))
#     plt.xticks(fontsize=12)
    plt.xlabel("Time")
    plt.ylabel("Trials")

In [ ]:
plt.figure(figsize=(24, 18))

plt.subplot(1, 2, 1)
plotRaster(20, Y_alltrials, spiketimes_alltrials, scale=1.0, spacing=2.0, flip=False)

plt.subplot(1, 2, 2) 
for t in range(0, 20): 
    plt.plot(cv.fits[0].infRes['post_mean'][t].T, label=sprintf("Trial %i", t+1))

plt.title("Latent State Trajectories")
plt.xlabel("Time")
plt.yticks([])
plt.legend(loc="best")

In [ ]:
cv.fits[0].plotTrajectory();

In [ ]:
cv.fits[0].plotParamSeq();

## Task 4. Visualization of covariance matrix.

Plot (a) the noise covariance matrix as well as its approximation using (b) one and (c) five latent variable(s). Use the analytical solution for the covariance matrix of the approximation*. Note that the solution is essentially the mean and covariance of the [log-normal distribution](https://en.wikipedia.org/wiki/Log-normal_distribution).

$ \mu = \exp(\frac{1}{2} \text{ diag}(CC^T)+d)$

$ \text{Cov}= \mu\mu^T \exp(CC^T)+\text{ diag}(\mu) - \mu\mu^T$ 

*[Krumin, M., and Shoham, S. (2009). Generation of Spike Trains with Controlled Auto- and Cross-Correlation Functions. Neural Computation 21, 1642–1664](http://www.mitpressjournals.org/doi/10.1162/neco.2009.08-08-847).

*Grading: 3 pts*

In [ ]:
fit_1 = cv.fits[0]
fit_5 = cv.fits[4]

fit_1.performSpikeCountAnalysis()
fit_5.performSpikeCountAnalysis()

**(a)** Plot noise covariance matrix

In [ ]:
# your plot here
noise_cov_obs = fit_1.E_yy_obs
noise_cov_1 = fit_1.E_yy_optim_params
noise_cov_5 = fit_5.E_yy_optim_params

vmin = np.min([noise_cov_obs, noise_cov_1, noise_cov_5])
vmax = np.max([noise_cov_obs, noise_cov_1, noise_cov_5])
vbounds = np.max([np.abs(vmin),np.abs(vmax)])

fig = plt.figure(figsize=(20,7))
plt.suptitle('Observed & Estimated Noise Covariance Matrices', fontsize=20)

plt.subplot(1,3,1)
plt.title("Observed Noise Covariance Matrix", fontsize=15)
plt.imshow(noise_cov_obs, vmin=-vbounds, vmax=vbounds)
plt.colorbar()
plt.tick_params(labelbottom=True, labeltop=False)

plt.subplot(1,3,2)
plt.title("Noise Covariance Matrix for 1 Latent Var", fontsize=15)
plt.imshow(noise_cov_1, vmin=-vbounds, vmax=vbounds)
plt.colorbar()
plt.tick_params(labelbottom=True, labeltop=False)

plt.subplot(1,3,3)
plt.title("Noise Covariance Matrix for 5 Latent Vars", fontsize=15)
plt.imshow(noise_cov_5, vmin=-vbounds, vmax=vbounds)
plt.colorbar()
plt.tick_params(labelbottom=True, labeltop=False)

In [ ]:
# # your plot here
# diff_obs_1 = np.abs(noise_cov_obs-noise_cov_1)
# diff_obs_5 = np.abs(noise_cov_obs-noise_cov_5)
# diff_1_5 = np.abs(noise_cov_1-noise_cov_5)

# vmin = np.min([diff_obs_1,diff_obs_5,diff_1_5])
# vmax = np.max([diff_obs_1,diff_obs_5,diff_1_5])
# vbounds = np.max([np.abs(vmin),np.abs(vmax)])

# plt.figure(figsize=(20,7))
# plt.suptitle('Absolute differences between noise covariance matrices',fontsize=20)

# plt.subplot(1,3,1)
# plt.imshow(diff_obs_1, vmin=-vbounds, vmax=vbounds)
# plt.colorbar()
# plt.title('|Observed - 1 latent var|',fontsize=15)
# plt.tick_params(labelbottom=True,labeltop=False)

# plt.subplot(1,3,2)
# plt.imshow(diff_obs_5, vmin=-vbounds, vmax=vbounds)
# plt.colorbar()
# plt.title('|Observed - 5 latent vars|',fontsize=15)
# plt.tick_params(labelbottom=True,labeltop=False)

# plt.subplot(1,3,3)
# plt.imshow(diff_1_5, vmin=-vbounds, vmax=vbounds)
# plt.colorbar()
# plt.title('|1 latent variable - 5 latent vars|',fontsize=15)
# plt.tick_params(labelbottom=True,labeltop=False)

### Analytical Approximations

In [ ]:
def mu(C,d):
    return np.exp((1/2)*np.diag(np.outer(C,C))+d)

def noise_cov_approx(C,d):
    mean = mu(C,d)
    return np.outer(mean,mean) * np.exp(np.outer(C,C))+ np.diag(mean)  - np.outer(mean,mean.T)

**(b)** Plot aproximation of noise covariance matrix using 1 latent variable

In [ ]:
C_est = C_est_matrix[:, 0]
Σ_est = noise_cov_approx(C_est, cv.fits[0].optimParams['d'])
plt.title("Analytically Approximated Noise Covariance Matrix for 1 Latent Var", fontsize=15)
plt.imshow(Σ_est)
plt.colorbar()
plt.tick_params(labelbottom=True, labeltop=False)

**(c)** Plot approximation of noise covariance matrix using 5 latent variables

In [ ]:
C_est = C_est_matrix[:, 4]
Σ_est = noise_cov_approx(C_est, cv.fits[4].optimParams['d'])
plt.title("Analytically Approximated Noise Covariance Matrix for 5 Latent Vars", fontsize=15)
plt.imshow(Σ_est)
plt.colorbar()
plt.tick_params(labelbottom=True, labeltop=False)